In [3]:
%cd D:\Laptop_Projects\LLMEval-1

D:\Laptop_Projects\LLMEval-1


In [4]:
from server.metrics.naturalness import NaturalnessHelper, get_meta_data, get_ranges
import server.metrics.stylistic as st
import json
import pandas as pd
import numpy as np
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
import plotly.express as px

In [5]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

In [6]:
%cd D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data
data_nat = json.load(open(r'final_metrics_naturalness.json'))
# data_natural = json.load(open(r'naturalness_float.json'))
# df = pd.DataFrame(data_natural)

# mean_sd_list,ranges = get_meta_data(df)
# print(ranges)
# print(mean_sd_list)
# nat_eval = st.NaturalnessEvaluator(mean_sd_list,ranges)

D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data


Run if you don't have the naturalness_combined.json

In [7]:
all_summ_list =[]
for dat in data_nat:
    writer_summ ={}
    LLM_summ = {}
    writer_summ['article_id'] = dat['article_id']
    writer_summ['summary'] = dat['writer_summary']
    writer_summ['summary_type'] = "Human"
    writer_keys = list(dat.keys())
    for key in writer_keys:
        if key.startswith('writer') or key.startswith('Writer'):
            new_key = key.replace('writer_', '').replace('writer', '').replace("Writer",'').strip()
            writer_summ[new_key] = dat[key]
    all_summ_list.append(writer_summ)

    LLM_summ['article_id'] = dat['article_id']
    LLM_summ['summary_type'] = "LLM"
    LLM_summ['summary'] = dat['text-davinci-002_summary']
    LLM_keys = list(dat.keys()) 
    for key in LLM_keys:
        if key.startswith('LLM'):
            new_key = key.replace('LLM_', '').replace('LLM', '').strip()
            LLM_summ[new_key] = dat[key]
    all_summ_list.append(LLM_summ)
# save_json(all_summ_list,r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\naturalness_combined.json')

Graphs for linguistic metrics used in naturalness

In [8]:
all_summ_list = json.load(open(r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\naturalness_combined.json'))
df_natural = pd.DataFrame(all_summ_list)

df_dep = df_natural.copy()
df_dep['count']=1
df_dep.loc[df_dep['summary_type']=="Human",'count'] =-1
fig = px.histogram(df_dep, x="Average Dependency tree heights", y="count", color='summary_type')
fig.show()

Line graph for frequency of average sentence lengths. LLM's tend to generate longer sentences with a higher frequency than humans.

In [47]:
df_sent = df_natural.copy()
grouped = df_sent.groupby(['summary_type', 'average_sentence_lengths']).size().reset_index(name='frequency_sent_length')
df = pd.merge(df_sent, grouped, on=['summary_type', 'average_sentence_lengths'], how='left')
df['frequency_sent_length'].fillna(0, inplace=True)
df['frequency_sent_length'] = df['frequency_sent_length'].astype(int)
df = df.sort_values(by=['average_sentence_lengths'])

fig = px.line(df, x="average_sentence_lengths", y="frequency_sent_length", color='summary_type')
fig.show()

Average left subtree height vs 

In [48]:
df_LLM = df_natural.loc[df_natural['summary_type']=="LLM"] 
df_human = df_natural.loc[df_natural['summary_type']=="Human"] 

In [49]:
fig = px.density_contour(df_human, x="avg_left_subtree_height", y="avg_right_subtree_height",text_auto=True)
fig.update_traces(contours_coloring="fill", contours_showlabels = True)
fig.show()

In [22]:
fig = px.density_contour(df_LLM, x="avg_left_subtree_height", y="avg_right_subtree_height",text_auto=True)
fig.update_traces(contours_coloring="fill", contours_showlabels = True)
fig.show()

In [51]:
df_dep = df_natural.copy()
df_dep['count']=1
df_dep.loc[df_dep['summary_type']=="Human",'count'] =-1
df_dep['average word lengths'] = df_dep['average word lengths'].astype(int)
fig = px.histogram(df_dep, x="average word lengths", y="count", color='summary_type')
fig.show()

In [23]:
fig = px.density_heatmap(df_human, x="avg_left_subtree_height", y="avg_right_subtree_height",text_auto=True)
fig.show()

In [24]:
fig = px.density_heatmap(df_LLM, x="avg_left_subtree_height", y="avg_right_subtree_height",text_auto=True)
fig.show()

In [53]:
y = df_natural['summary_type']
X_unscaled = df_natural.drop(columns = ['article_id','summary','summary_type'])

In [54]:
from sklearn.preprocessing import StandardScaler
encoder = LabelEncoder()
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X_unscaled))
X.columns = X_unscaled.columns
y = [str(value) for value in y]
y = encoder.fit_transform(y)

In [57]:
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(X)
importance = pca.components_[0]
feature_importance = pd.DataFrame(list(zip(X.columns, importance)), columns=['Feature', 'Importance'])
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(feature_importance)

                           Feature  Importance
1  Average Dependency tree heights    0.504439
6          avg_left_subtree_height    0.486318
7         avg_right_subtree_height    0.441374
2         average_sentence_lengths    0.427449
9               num_right_subtrees    0.252828
8                num_left_subtrees    0.158020
5         average_total_arc_length    0.137703
4         average_right_arc_length    0.128035
0             average word lengths   -0.023296
3          average_left_arc_length   -0.081986


In [58]:
feature_cols = [
                # 'average word lengths', 
                'Average Dependency tree heights',
                'average_sentence_lengths', 
                # 'average_left_arc_length',
                # 'average_right_arc_length', 
                # 'average_total_arc_length',
                'avg_left_subtree_height', 
                'avg_right_subtree_height',
                # 'num_left_subtrees', 
                # 'num_right_subtrees'
                ]

X_train, X_test, y_train, y_test = train_test_split(X[feature_cols],y, test_size=0.25, random_state=42) # change to feature_LLM for LLM weights
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_preds = clf.predict(X_test)
acc = accuracy_score(y_test, y_preds)
print (acc)

importances = clf.feature_importances_
importances.sort()
for feature, importance in zip(feature_cols, importances):
    print(f"{feature}: {importance}")

0.9933333333333333
Average Dependency tree heights: 0.20551969186015231
average_sentence_lengths: 0.23828211739802285
avg_left_subtree_height: 0.24983966373071514
avg_right_subtree_height: 0.3063585270111097


Naturalness for custom text

Calculate Naturalness for whole dataset

In [61]:
all_summ_list = json.load(open(r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\naturalness_scores_float.json'))
meta_data = json.load(open(r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\meta_data.json'))
df_nat_bin = pd.DataFrame(all_summ_list)

In [66]:
mean_sd_list = meta_data["naturalness"]["mean_sd"]
mean_sd = []
for range in mean_sd_list:
    mean_sd.append([range[0],range[1]])
print(mean_sd)
ranges=[]
classes = meta_data["naturalness"]["ranges"]
for cl in classes:
    classes.append([cl[0],cl[1]])
print(classes)

[[3.909472724370619, 0.5866325558455203], [19.792292710072314, 4.383204361989693], [3.224778530155862, 0.6115396533323255], [2.868492737029371, 0.6894262704576944]]


In [5]:
mean_sd_list,ranges = get_meta_data(df_nat_bin)
nat_eval = st.NaturalnessEvaluator(mean_sd_list,ranges)
for i,datum in enumerate(all_summ_list):
    print('{}/{}'.format(i, len(all_summ_list)))
    datum['naturalness_score'],datum['naturalness_score_class'] = nat_eval.default(datum['summary'])
    # datum['naturalness_score_writer'] = nat.default(datum['writer_summary'])
save_json(all_summ_list,r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\naturalness_scores_binary.json')
# data_nat_df = pd.DataFrame(data_nat)
# data_nat_df.to_csv('naturalness_std.csv', index = True) 

0/1198
1/1198
2/1198
3/1198
4/1198
5/1198
6/1198
7/1198
8/1198
9/1198
10/1198
11/1198
12/1198
13/1198
14/1198
15/1198
16/1198
17/1198
18/1198
19/1198
20/1198
21/1198
22/1198
23/1198
24/1198
25/1198
26/1198
27/1198
28/1198
29/1198
30/1198
31/1198
32/1198
33/1198
34/1198
35/1198
36/1198
37/1198
38/1198
39/1198
40/1198
41/1198
42/1198
43/1198
44/1198
45/1198
46/1198
47/1198
48/1198
49/1198
50/1198
51/1198
52/1198
53/1198
54/1198
55/1198
56/1198
57/1198
58/1198
59/1198
60/1198
61/1198
62/1198
63/1198
64/1198
65/1198
66/1198
67/1198
68/1198
69/1198
70/1198
71/1198
72/1198
73/1198
74/1198
75/1198
76/1198
77/1198
78/1198
79/1198
80/1198
81/1198
82/1198
83/1198
84/1198
85/1198
86/1198
87/1198
88/1198
89/1198
90/1198
91/1198
92/1198
93/1198
94/1198
95/1198
96/1198
97/1198
98/1198
99/1198
100/1198
101/1198
102/1198
103/1198
104/1198
105/1198
106/1198
107/1198
108/1198
109/1198
110/1198
111/1198
112/1198
113/1198
114/1198
115/1198
116/1198
117/1198
118/1198
119/1198
120/1198
121/1198
122/1198
123

In [13]:
meta_data = json.load(open(r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\meta_data.json'))
min_max = meta_data["naturalness"]["naturalness_min_max"]
min_max

[-1.815013403616553, 2.4030803895301096]

In [14]:
all_summ_list = json.load(open(r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\naturalness_scores_binary.json'))
for summ in all_summ_list:
    summ['naturalness_score'] = 1-(summ['naturalness_score']-min_max[0])/(min_max[1]-min_max[0])
all_summ_list

[{'article_id': '18cba9a8f2f64055a707452638182303',
  'summary': 'The mayor of Baltimore fired the police chief and replaced him with his deputy. According to the mayor, crime in the city was unacceptable. Riots in the city after a man died in police custody and a surge in homicide rates were cited as reasons for the firing.   ',
  'summary_type': 'Human',
  'average word lengths': 4.354166666666667,
  'Average Dependency tree heights': 3.5283018867924527,
  'average_sentence_lengths': 16.0,
  'average_left_arc_length': 2.2,
  'average_right_arc_length': 2.7,
  'average_total_arc_length': 2.5,
  'avg_left_subtree_height': 2.95,
  'avg_right_subtree_height': 2.5,
  'num_left_subtrees': 20,
  'num_right_subtrees': 30,
  'naturalness_score': 0.7156745588358897,
  'naturalness_score_class': 'avg'},
 {'article_id': '18cba9a8f2f64055a707452638182303',
  'summary_type': 'LLM',
  'summary': 'The mayor of Baltimore has sacked the city\'s police chief, saying his leadership has become a distract

In [16]:
save_json(all_summ_list,r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\naturalness_scores.json')

In [10]:
df = pd.DataFrame(all_summ_list)
df['naturalness_score_class'].value_counts()
df['naturalness_score'] = 1-(df['naturalness_score']-df['naturalness_score'].min())/(df['naturalness_score'].max()-df['naturalness_score'].min())
df

,article_id,summary,summary_type,average word lengths,Average Dependency tree heights,average_sentence_lengths,average_left_arc_length,average_right_arc_length,average_total_arc_length,avg_left_subtree_height,avg_right_subtree_height,num_left_subtrees,num_right_subtrees,naturalness_score,naturalness_score_class
0,18cba9a8f2f64055a707452638182303,The mayor of Baltimore fired the police chief ...,Human,4.354167,3.528302,16.000000,2.200000,2.700000,2.500000,2.950000,2.500000,20,30,0.715675,avg
1,18cba9a8f2f64055a707452638182303,The mayor of Baltimore has sacked the city's p...,LLM,5.181818,3.865385,21.000000,1.666667,5.130435,3.260000,3.185185,2.739130,27,23,0.575267,low
2,66f39853ad2b437c8bdca86ae74bb35f,Morocco joined the African Union after a refer...,Human,4.807692,3.250000,13.000000,1.411765,5.000000,2.653846,2.764706,1.777778,17,9,0.871484,good
3,66f39853ad2b437c8bdca86ae74bb35f,The article discusses Western Sahara's reactio...,LLM,4.786885,4.015152,19.000000,1.346154,4.432432,3.158730,3.692308,2.783784,26,37,0.534783,low
4,302c800172da420f9e2e80474a9cf5ec,James Haskell is a rugby player for the London...,Human,4.380000,4.175439,16.666667,1.954545,3.281250,2.740741,2.954545,3.625000,22,32,0.534377,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,2c80f9196b654048b01397ebd52d3518,"The Coronation Fountain in Welwyn Garden City,...",LLM,4.804348,3.730769,22.000000,1.909091,3.857143,3.000000,3.227273,2.535714,22,28,0.590913,low
1194,f1d84317501a4ba285f0e81384471f6e,The country of Scotland is potentially facing ...,Human,5.000000,3.208333,14.000000,1.782609,2.818182,2.288889,2.521739,2.181818,23,22,0.843025,good
1195,f1d84317501a4ba285f0e81384471f6e,The article discusses how sport funding in Sco...,LLM,4.851852,3.677419,26.000000,2.000000,8.615385,4.866667,3.176471,2.230769,17,13,0.580838,low
1196,0f1d41fcf8934fdf8fc993851ba9c6c4,"Leeds United has had a chaotic season, culmina...",Human,4.519231,3.327586,17.000000,1.392857,3.629630,2.490909,2.714286,2.185185,28,27,0.775452,good


In [36]:
Q1 = np.percentile(df['naturalness_score'], 25)
Q2 = np.percentile(df['naturalness_score'], 50)
Q3 = np.percentile(df['naturalness_score'], 75)
print("Q1", Q1)
print("Q2", Q2)
print("Q3", Q3)
# define ranges, add another range for all quartiles.
range1 = (df['naturalness_score'].min(), Q1)
range2 = (Q1,Q2)
range3 = (Q2,Q3)
range4 = (Q3, df['naturalness_score'].max())

print(f"Range 1: {range1}")
print(f"Range 2: {range2}")
print(f"Range 3: {range3}")
print(f"Range 4: {range4}")

Q1 0.4352518900573824
Q2 0.607625718803343
Q3 0.7156745588358897
Range 1: (0.0, 0.4352518900573824)
Range 2: (0.4352518900573824, 0.607625718803343)
Range 3: (0.607625718803343, 0.7156745588358897)
Range 4: (0.7156745588358897, 1.0)


In [28]:
df_l = df.loc[df_natural['summary_type']=="LLM"] 
df_h = df.loc[df_natural['summary_type']=="Human"] 
c1 = df_l['naturalness_score_class'].value_counts()
c2 = df_h['naturalness_score_class'].value_counts()
print("LLM: ",c1)
print("Human: ",c2)

LLM:  bad     184
avg     172
low     146
good     97
Name: naturalness_score_class, dtype: int64
Human:  good    199
low     149
avg     131
bad     120
Name: naturalness_score_class, dtype: int64
